# A Jupyter Kernel for Logtalk

This notebook provides an overview of the functionality and implementation of the Logtalk Jupyter kernel.

Running this notebook assumes Logtalk is installed using one of the provided installers or by running the manual installation script.

The default backend can be changed in the fly by adding a code cell at the top and running one of the following queries: `eclipse`, `gnu`, `sicstus`, `swi`, `trealla`, `xvm`, or `yap` (assuming that all these backend Prolog systems are installed). The default backend can be set for all notebooks in a directory by using a `logtalk_kernel_config.py` file (see the [logtalk-jupyter-kernel](https://github.com/LogtalkDotOrg/logtalk-jupyter-kernel) repo for details). If this file is not present, the default backend is SWI-Prolog.

This notebook is currently running using:

In [ ]:
%versions

## Execute Queries

Code cells can contain queries, multiple terms to be interpreted as directives and clauses to be added to a file, or code to be highlighted. By default, the contents of a code cell is interpreted as a query. For example:

In [ ]:
current_logtalk_flag(version_data, VersionData).

In [ ]:
X = [1,2,3], list::append(X, [4,5,6], Z).

We can also write multiple queries in a single cell:

In [ ]:
current_logtalk_flag(unicode, Unicode).
current_logtalk_flag(encoding_directive, EncodingDirective).

## Access and Reuse Query Bindings

The variable bindings from previous queries can be accessed by using the `%bindings` line magic:

In [ ]:
%bindings

Variable bindings can be reused using the corresponding `$Var` term. For example:

In [ ]:
forall(list::member(X, $Z), write(X)).


We can also print the previous queries using the `%queries` line magic:

In [ ]:
%queries

## Define Predicates

Predicates can be defined in `user` by using the `%%user` cell magic in the first line of a code cell:

In [ ]:
%%user

my_append([], Res, Res).
my_append([H|T], List, [H|Res]) :-
  my_append(T, List, Res).

Running a code cell defining predicates adds them to the database of the running kernel, making them available for use in queries:

In [ ]:
my_append([1,2], [3,4], R).

Predicate definitions can be split in multiple cells. For example:

In [ ]:
%%user

fact(a).
fact(b).

In the next cell, the previous clauses are replaced by new ones:

In [ ]:
%%user

fact(c).
fact(d).

In [ ]:
findall(X, fact(X), L).

But new clauses can be added instead by declaring a predicate *discontiguous* and later using instead the `%%user+` cell magic (think of `+` as meaning append):

In [ ]:
%%user

:- discontiguous(a/1).
a(1).
a(2).

In [ ]:
%%user+

b(3).
a(4).

In [ ]:
listing(a/1), listing(b/1).

## Define Objects, Protocols, and Categories

Logtalk entities are preferably defined using the `%%file FILENAME` or `%%file+ FILENAME` cell magic (the `+` variant appends to an existing file instead of redefining it). For example:

In [ ]:
%%file foo.lgt

:- object(foo).

    :- public(bar/0).
    bar :-
        write('Hello world!\n').

:- end_object.

Running a code cell defining Logtalk entities adds them to the database of the running kernel, making them available for use in queries:

In [ ]:
foo::bar.

## Highlight Code

Sometimes we want to simply display some code in a cell, benefiting from syntax highlight. This can be accomplished by using the `%%highlight` cell magic. For example:

In [ ]:
%%highlight

:- object(hello_world).

	% the initialization/1 directive argument is automatically executed
	% when the object is loaded into memory:
	:- initialization(write('Hello World!\n')).

:- end_object.

## Handling Multiple Solutions

As a notebook code cell doesn't provide the same interactive features of a traditional top-level interpreter, we can either ask for the next solution for a query in the next cell or, if practical/feasible, ask for all solutions at once.

In [ ]:
list::member(M, [a,b,c]).

In [ ]:
jupyter::retry.

We can also simply type:

In [ ]:
retry.

In altenative:

In [ ]:
findall(M, list::member(M, [a,b,c]), L).

See also the `%%table` cell magic below as an alternative to get and report all solutions for a goal.

## Debugging

The Logtalk `debugger` tool is loaded by default when we start a kernel. To illustrate, we load an example, compiling it in debug mode:

In [ ]:
set_logtalk_flag(debug, on), {elephants(loader)}.

Try one of examples queries:

In [ ]:
fred::number_of_legs(N).

Debugging cannot be performed interactively. But we can print the trace of a goal:

In [ ]:
debugger::trace, fred::number_of_legs(N), debugger::notrace.

We can also use spy points to narrow the debugging output. For example:

In [ ]:
{planets(loader)}, debugger::spy(gravitational_acceleration/1), mars::weight(m2, W2).

Turn off debugging and compiling in debug mode:

In [ ]:
debugger::nodebug, set_logtalk_flag(debug, off).

## Running Tests

Tests can be run by loading their driver file. For example:

In [ ]:
{ack(tester)}.

## Benchmarking Capabilities

Whenever a query is executed, its runtime is stored in the database and can be accessed immediately after:

In [ ]:
list::member(M, [1,2,3]).

In [ ]:
jupyter::print_query_time.

## Structured Output

Query bindings can be displayed in a table with a row per solution and a column per variable binding by using the `%%table` cell magic:

In [ ]:
%%table
list::member(Number, [10,20,30,40]), Square is Number*Number.

Columns for variables whose name starts with an underscore are omitted. For example:

In [ ]:
%%table
_List = [10,20,30,40], list::nth1(Position, _List, Element).

To print a table but also save it to a CSV or TSV file, use instead the `%%csv file.csv` or `%%tsv file.tsv` cell magics. For example:

In [ ]:
%%tsv flags.tsv
current_logtalk_flag(Flag, Value).

## Data Visualization

Data visualization is available by using the `%%data` cell magic and a goal that binds a variable named `Data` or `_Data` to a list of pairs. The `type` key is required. The other keys depend on the type of visualization. Currently, data visualization can use the `matplotlib.pyplot` plots illustrated next. See the `matplotlib` documentation for the details on plot specific keys. Multiple subplots are not currently supported.

- The supported plot keys are `suptitle`, `title`, `xlabel`, `ylabel`, `bar_label`, `xscale`, `yscale`, `xticks`, `yticks`, `xlim`, `ylim`, `margins`, `rc`, `grid`, `thetagrids`, `rgrids`, `autoscale`, `tight_layout`, `legend`, `annotate`, `text`, and `figtext`.
- The values of the `suptitle`, `title`, `xlabel` and `ylabel` keys can be either an atom or a list of pairs (in which case, there must be a pair with a `label` key).
- The value of the `rc` key must be a list of pairs that includes a pair with a `label` key.
- The value of the `annotate` key must be a list of pairs that includes pairs with `text` and `xy` keys.
- The values of the `text` and `figtext` keys must be a list of pairs that includes pairs with `x`, `y`, and `s` keys.
- The values of the `xticks`, `yticks`, `xlim`, `ylim`, `margins`, `grid`, `thetagrids`, `rgrids`, `autoscale`, `tight_layout`, `legend`, and `bar_level` keys must be a list of pairs.
- The atoms `true`, `false`, and `none` can be used to represent the corresponding Python `True`, `False`, and `None` values for plot keywords that require them.

In [ ]:
%%data
Data = [type-bar, title-'Bar Graph', x-[10, 20, 30, 40, 50, 60], height-[13, 45, 23, 34, 96, 76], color-dodgerblue, width-5, text-[x-15, y-85, s-'Blue bars!', fontdict-[color-mediumblue, style-italic, size-large]]].

In [ ]:
%%data
Data = [type-barh, title-[label-'Horizontal bar graph', fontsize-18], y-['Apples','Oranges','Pears','Mangos'], width-[1,4,9,16], color-[orangered,orange,burlywood,khaki], bar_label-[label_type-center]]

In [ ]:
%%data
Data = [type-pie, title-'Pie Graph', x-[35, 20, 30, 40, 50, 30], labels-['Apple', 'Bananna', 'Grapes', 'Orange', 'PineApple', 'Dragon Fruit'], autopct-'%.2f%%'].

In [ ]:
%%data
logtalk_load(random(loader)),
random::sequence(1000, -20, 20, _List),
_Data = [type-hist, title-'Histogram', x-_List, bins-20, color-skyblue, edgecolor-black, xlabel-[label-'Values', loc-right, color-brown], ylabel-[label-'Frequency', loc-top, color-brown]].

In [ ]:
%%data
Data = [type-scatter, title-'Scatter plot', x-[5,7,8,7,2,17,2,9,4,11,12,9,6], y-[99,86,87,88,111,86,103,87,94,78,77,85,86], grid-[color-navy, linestyle-'-', linewidth-0.2]].

In [ ]:
%%data
Data = [type-plot, title-'Line plot', x-[1,2,3,4], y-[1,4,9,16], xticks-[ticks-[1,2,3,4], labels-[a,b,c,d]], annotate-[text-'Interesting value!', xy-[2,4], xytext-[3,4], arrowprops-[width-1, headwidth-4, facecolor-black, shrink-0.05]]].

In [ ]:
%%data
Data = [type-loglog, title-'Log line plot', x-[1,2,3,4], y-[1,4,9,16]].

In [ ]:
%%data
logtalk_load(types(loader)),
integer::sequence(0, 360, 10, _Xs),
findall(Y, (list::member(X,_Xs), Y is sin((X*pi)/180)), _Ys),
_Data = [type-stem, title-'Stem plot', x-_Xs, y-_Ys, xlabel-'X (degrees)', ylabel-'sin(X)'].

In [ ]:
%%data
logtalk_load(random(loader)),
random::randseq(100, 0.0, 100.0, _Positions),
_Data = [type-eventplot, title-'Event plot', positions-_Positions, orientation-vertical, linelengths-1.4, color-cyan].

In [ ]:
%%data
Data = [type-step, title-'Step plot', x-[1, 2, 3, 4, 5], y-[0, 1, 0, 2, 1], xlabel-'Values', ylabel-'Frequency'].

In [ ]:
%%data

% List of Days
Days = [1, 2, 3, 4, 5], 
% Number of Study Hours
Studying = [7, 8, 6, 11, 7],
% Number of Playing Hours
Playing = [8, 5, 7, 8, 13],
% Stack plot with X, Y, colors value
Data = [type-stackplot, title-'Stack plot', x-Days, y-[Studying, Playing], labels-['Studying', 'Playing'], legend-[loc-'upper left'], colors-[orange, cyan], xlabel-'Days', ylabel-'No of Hours'].

In [ ]:
%%data
logtalk_load(random(loader)),
findall(X, (integer::between(1,200,_), backend_random::random(80.0, 120.0, X)), _Xs),
findall(Y, (integer::between(1,200,_), backend_random::random(60.0, 90.0, Y)), _Ys),
findall(Z, (integer::between(1,200,_), backend_random::random(75.0, 105.0, Z)), _Zs),
_Data = [type-boxplot, title-'Box plot', x-[_Xs,_Ys,_Zs], positions-[2, 4, 6], widths-1.5, patch_artist-true, showmeans-false, showfliers-false].

In [ ]:
%%data
Data = [type-errorbar, title-'Error bar', x-[2,4,6], y-[3.6,5.0,4.2], yerr-[0.9,1.2,0.5], fmt-o, linewidth-2, capsize-6, xlim-[left-0, right-8], xticks-[ticks-[1,2,3,4,5,6,7,8]], ylim-[bottom-0, top-8], yticks-[ticks-[1,2,3,4,5,6,7,8]]].

In [ ]:
%%data
logtalk_load(random(loader)),
findall(X, (integer::between(1,5000,_), backend_random::random(-2.0, 2.0, X)), _Xs),
findall(Y, (list::member(X,_Xs), backend_random::random(-3.0, 3.0, Z), Y is 1.2*X + Z/3), _Ys),
_Data = [type-hexbin, title-'Hexbin plot', x-_Xs, y-_Ys, gridsize-20, xlim-[left-(-2), right-2], ylim-[bottom-(-3), top-3]].

In [ ]:
%%data
logtalk_load(random(loader)),
findall(X, (integer::between(1,5000,_), backend_random::random(X1), X1 =\= 0.0, backend_random::random(X2), X is sqrt(-2.0 * log(X1)) * cos(2.0*pi*X2)), _Xs),
findall(Y, (list::member(X,_Xs), backend_random::random(Z1), Z1 =\= 0.0, backend_random::random(Z2), Z is sqrt(-2.0 * log(Z1)) * cos(2.0*pi*Z2), Y is 1.2*X + Z/3), _Ys),
float::sequence(-3.0,3.0,0.1,_Range,_),
_Data = [type-hist2d, title-'Hist2d plot', x-_Xs, y-_Ys, bins-[_Range, _Range], xlim-[left-(-2), right-2], ylim-[bottom-(-3), top-3]].

In [ ]:
%%data
logtalk_load(random(loader)),
findall(X, (integer::between(1,5000,_), backend_random::random(-2.0, 2.0, X)), _Xs),
_Data = [type-ecdf, title-'ECDF plot', x-_Xs].

In [ ]:
%%data
logtalk_load(random(loader)),
integer::sequence(-2, 2, _Is),
findall(F, (list::member(I,_Is), F is cos(I**2)), _Fs),
Data = [type-polar, title-[label-'Polar plot', fontweight-bold], theta-_Is, r-_Fs].

## Input Widgets

**Experimental.** Widgets are rendered in the notebook using HTML/JavaScript. Currently, the kernel uses a web server to handle the widget callbacks that update the widget state. This webserver is started automatically by the kernel. By default, it uses `127.0.0.1` as the IP address and the first available port in the range 8900-8999. The IP and port number can be queried using the `jupyter_widgets::webserver/2` predicate (e.g., to set up port forwarding). Both IP and port range can be customized in the kernel configuration file, `logtalk_kernel_config.py`, saved in the same directory as the notebooks.

The `jupyter_widgets` object provide a set of predicates for creating and deleting input widgets, querying the widgets values, and debugging widgets. The widget identifiers (atoms) must be unique. When a code cell that creates a widget may be run repeatedly, delete the widget (using the `remove_widget/1` or `remove_all_widgets/0` predicates) before recreating it.

### Text Input Widget

In [ ]:
jupyter_widgets::create_text_input(name_input, 'Enter your name:', 'John Doe').

In [ ]:
jupyter_widgets::get_widget_value(name_input, Name).

### Password Input Widget

In [ ]:
jupyter_widgets::create_password_input(password_input, 'Enter your password:').

In [ ]:
jupyter_widgets::get_widget_value(password_input, Name).

### Number Input Widget

In [ ]:
jupyter_widgets::create_number_input(age_input, 'Enter your age:', 0, 120, 1, 25).

In [ ]:
jupyter_widgets::get_widget_value(age_input, Age).

In [ ]:
jupyter_widgets::create_number_input(x_input, 'Enter x:', 0.0, 10.0, 0.02, 5.0).

In [ ]:
jupyter_widgets::get_widget_value(x_input, X).

### Slider Widget

In [ ]:
jupyter_widgets::create_slider(temperature_slider, 'Temperature (°C)', -10, 40, 5, 20).

In [ ]:
jupyter_widgets::get_widget_value(temperature_slider, Temperature).

In [ ]:
jupyter_widgets::create_slider(pressure_slider, 'Pressure (kPa)', -10.5, 25.5, 0.1, 18.0).

In [ ]:
jupyter_widgets::get_widget_value(pressure_slider, Pressure).

### Date Widget

In [ ]:
jupyter_widgets::create_date_input(birth_date_input, 'Enter your birth date:', '1990-01-01').

In [ ]:
jupyter_widgets::get_widget_value(birth_date_input, BirthDate).

### Time Widget

In [ ]:
jupyter_widgets::create_time_input(meeting_time_input, 'Enter meeting time:', '14:00').

In [ ]:

jupyter_widgets::get_widget_value(meeting_time_input, MeetingTime).

### Email Widget

In [ ]:
jupyter_widgets::create_email_input(email_input, 'Enter your email:', 'john.doe@example.com', '.+@.+\\..+').

In [ ]:
jupyter_widgets::get_widget_value(email_input, Email).

### URL Widget

In [ ]:
jupyter_widgets::create_url_input(url_input, 'Enter a URL:', 'https://www.example.com', 'https?://.+').

In [ ]:
jupyter_widgets::get_widget_value(url_input, URL).

### File Widget

In [ ]:
jupyter_widgets::create_file_input(file_input, 'Select a file:').

In [ ]:
jupyter_widgets::get_widget_value(file_input, File).

### Color Widget

In [ ]:
jupyter_widgets::create_color_input(color_input, 'Choose a color:', '#ff0000').

In [ ]:
jupyter_widgets::get_widget_value(color_input, Color).

### Dropdown Widget

Create a dropdown selection:

In [ ]:
jupyter_widgets::create_dropdown(color_select, 'Choose a color:', [red, green, blue, yellow, purple]).

In [ ]:
jupyter_widgets::get_widget_value(color_select, Color).

### Checkbox Widget

In [ ]:
jupyter_widgets::create_checkbox(newsletter_checkbox, 'Subscribe to newsletter', false).

In [ ]:
jupyter_widgets::get_widget_value(newsletter_checkbox, Color).

### Button Widget

In [ ]:
jupyter_widgets::create_button(action_button, 'Click Me!').

In [ ]:
jupyter_widgets::get_widget_value(action_button, Clicked).

### Generic Widget

The `jupyter_widgets::create_input/3` predicate allows creating a generic HTML input widget with custom attributes. Consult a HTML reference for the available attributes per input type.

In [ ]:
jupyter_widgets::create_input(widget_id, 'Enter a salutation:', [type-text, placeholder-'Hello World!', style-'font-size: 20px; color: blue;']).

In [ ]:
jupyter_widgets::get_widget_value(widget_id, Value).

### List all Widgets

In [ ]:
jupyter_widgets::widgets.

## Input Forms

**Experimental.** For more complex data collection, we can use input forms instead of individual widgets. Forms are rendered in the notebook using HTML/JavaScript and use the same callback web server used by the widgets to handle the form submissions.

The `jupyter_forms` object provides predicates for creating and managing HTML forms. The form identifiers (atoms) must be unique. When a code cell that creates a form may be run repeatedly, delete the form (using the `remove_form/1` or `remove_all_forms/0` predicates) before recreating it.

Create a form with multiple field types:

In [ ]:
jupyter_forms::create_input_form(contact_form, [
    text_field(name, 'Full Name:', 'John Doe'),
    email_field(email, 'Email Address:', 'jdoe@example.com', '.+@.+\\..+'),
    number_field(age, 'Age:', 0, 120, 1, 20),
    dropdown_field(country, 'Country:', [portugal, usa, canada, uk, germany, france]),
    textarea_field(message, 'Message:', '', 4),
    checkbox_field(newsletter, 'Subscribe to newsletter:', false)
], [
    title('Contact Information'),
    submit_label('Submit Form'),
    cancel_label('Clear Form')
]).

Retrieve the form data after submission:

In [ ]:
jupyter_forms::get_form_data(contact_form, ContactData).

Create a form that collects survey data:

In [ ]:
jupyter_forms::create_input_form(survey_form, [
    text_field(participant_id, 'Participant ID:', ''),
    dropdown_field(experience, 'Programming Experience:', [beginner, intermediate, advanced]),
    number_field(years_coding, 'Years of Coding:', 0, 50, 1, 1),
    dropdown_field(favorite_language, 'Favorite Language:', [python, java, javascript, prolog, logtalk]),
    textarea_field(comments, 'Additional Comments:', '', 3)
], [
    title('Programming Survey'),
    submit_label('Submit Survey')
]).

In [ ]:
% Process survey results
jupyter_forms::get_form_data(survey_form, SurveyData).

## Printing Terms

Complex compound terms can be displayed as a tree by using the `%%tree` cell magic:

In [ ]:
%%tree
a(1, b(2, c(3, 4))).

## Cell and Line Magic

Help on available cell and line magic can be printed using either the `jupyter::magic` query or by using the `%magic` line magic:

In [ ]:
%magic

## Introspection

- Various `jupyter` predicates
    - Access documentation with a help predicate

In [ ]:
jupyter::help.

- Various `juypter` (mostly convenience) predicates
- Difficult to remember all of them
    - In addition to completion and inspection: predicate to print all documentation

## Jupyter

- Originates from the **IPython** project
    - Enables interactive Python development
    - Several frontends, including a former version of **Jupyter Notebook**
        - Web application for handling Jupyter notebooks
        - Planned to be replaced by **JupyterLab**



- *Two-process model*:

    - Client process: responsible for user interaction
    - Kernel process: handles code execution

## Architecture






Kernel split in three:
- Extends IPython kernel: **inherits** the communication with a frontend via the ZeroMQ protocol


- Does not interpret Logtalk itself
    - Starts an existing Logtalk instance in a **subprocess**
        - Communicates with it according to the JSON-RPC 2.0 protocol
    - For any code execution **request**:
        - Sends a request message to the Logtalk server containing the **code**
        - Terms are read from the code and handled
    
    
- Make the kernel **extensible**: additional layer of a *kernel implementation* in between
    - **Responsible** for basically all functionality (e.g. handling Logtalk **server**)
    - For every request the kernel receives, a **method** of the implementation class is called
    - Kernel started: loads **config** file
        - Can contain paths to interpreter-specific Python class files
    - By **extending** default implementation class and **overriding** methods
        - Kernel behaviour can be adjusted
    - Had to be done to support predicate inspection for both    
- Configure to start a different Logtalk server

## Changing the Prolog Backend

- Switch between Prolog backends on the fly
- The previous server process is kept running
    - When switching back, the database state has not changed

Several Prolog backends are supported and shortcuts are provided to switch to them if installed:

- ECLiPSe (`eclipse`)
- GNU Prolog (`gnu`)
- SICStus Prolog (`sicstus`)
- SWI-Prolog (`swi`)
- Trealla Prolog (`trealla`)
- XVM (`xvm`)
- YAP (`yap`)

The above shortcuts assume Logtalk was installed using either one of the provided installers or by running the manual installation script (i.e. you can run e.g. Logtalk with SWI-Prolog by simply typing `swilgt` on a POSIX system or `swilgt.ps1` on a Windows system). Alternatively, that you're running Logtalk from a git clone directory with the `LOGTALKHOME` and `LOGTALKUSER` environment variables defined and pointing to the clone directory (i.e. you can run e.g. Logtalk with SWI-Prolog by simply typing `swilgt.sh` on a POSIX system). But you can always switch Prolog backends using the `jupyter::set_prolog_backend(BackendIntegrationScript)` predicate instead (e.g. `jupyter::set_prolog_backend('swilgt.sh')`).

## Extending the Kernel

- Original Prolog only kernel was developed for SICStus Prolog and later extended to SWI-Prolog
- Current Logtalk kernel supports those and other Prolog backends (including ECLiPSe, GNU Prolog, Trealla Prolog, XVM, and YAP)
- Portable code except for the non-standard stream redirection details that depend on the backend

    

- By **replacing the Logtalk server**, the Python part can easily support a different implementation
    - Requirements: receive requests as JSON-RPC 2.0 messages, handle them, and send responses
    - Might be possible to further extend the existing server with conditional compilation
        - Advanced features might require significant changes

- By **overriding the `LogtalkKernelBaseImplementation` class**, most of the basic kernel behaviour can be adjusted

- Server replacement:
    - Extend existing:
        - Implementing **basic code execution** should not require major effort
        - More **advanced features** might involve significant changes

- Replacement of the server does not suffice?: Python extension

## Future Work

- Support other Prolog backends (waiting on requests to their maintainers for missing functionality)

- Combine strengths of several Prolog backends
    - Kernel can be connected with multiple servers at once
    - Reusing results for another one should be relatively easy

- Send commands to all available Logtalk servers *at once*
    - Detect differences in the behaviour
    - Compare the performance by using the benchmarking functionality